<a href="https://colab.research.google.com/github/uvezero/Book-Recommendation-KNN/blob/main/health_costs_with_regression_DNN_multipleinput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.compat.v2.feature_column as fc
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers.experimental import preprocessing
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from IPython.display import clear_output


In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

First lets check out the data and analyze it.

In [ ]:
dataset.age.hist(bins=10,color='turquoise')

In [ ]:
dataset.children.value_counts().plot(kind='barh',color='crimson')

In [ ]:
dataset.sex.value_counts().plot(kind='barh',color='rosybrown')

In [ ]:
dataset.region.value_counts().plot(kind='barh',color='navy')

In [ ]:
dataset.smoker.value_counts().plot(kind='barh',color='k')

In [ ]:
dataset.expenses.hist(bins=30,color='palegreen')

In [ ]:
dataset.describe().transpose() #This needs Normalization

In [ ]:
# lets encode smoker, region and sex columns
encoder = LabelEncoder()
dataset['sex']=encoder.fit_transform(dataset['sex'])
dataset['smoker']=encoder.fit_transform(dataset['smoker'])
dataset['region']=encoder.fit_transform(dataset['region'])
dataset.tail()

In [ ]:
#lets split dataset intro train(80%) and test(20%)
train_dataset=dataset.sample(frac=0.8,random_state=25)
test_dataset=dataset.drop(train_dataset.index)
print(test_dataset.shape[0], train_dataset.shape[0])

#pop expenses column
train_labels=train_dataset.pop('expenses')
test_labels=test_dataset.pop('expenses')

In [ ]:
def plot(feature, x=None, y=None):
  plt.figure(figsize=(7,5))
  plt.scatter(train_dataset[feature],train_labels,label='Data')
  if x is not None and y is not None:
    plt.plot(x,y,color='k', label='Predictions')
  plt.xlabel(feature)
  plt.ylabel('expenses')
  plt.title(feature)
  plt.legend()
plot('bmi') , plot('age') #this output is not very intuitive but we are going to build the model for multiple inputs so we dont ened to choose a specific label as input

In [ ]:
print(train_dataset.describe().transpose()[['mean','std']])

In [ ]:
#Normalize
normalizer = tf.keras.layers.Normalization(axis=-1)
#adapt to our dataset
normalizer.adapt(np.array(train_dataset)) # generalised normalizer
print(normalizer.mean.numpy())

Normalization = (𝑌 - <𝑌>)/σ



In [ ]:
#Build the Linear Regression with a DNN(two hidden layers) for multiple inputs (generalised normalizer)
model= tf.keras.Sequential([
    normalizer,
    layers.Dense(64,activation='relu'),
    layers.Dense(32,activation='relu'),
    layers.Dense(units=1) #Linearise

])
model.summary()

In [ ]:
model.predict(train_dataset)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss=keras.losses.MeanAbsoluteError(),
    metrics=['mae','mse'])

In [ ]:
history= model.fit(
    train_dataset,
    train_labels,
    verbose=1,
    epochs=100,
    validation_split=0.2)
clear_output()

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
